In [51]:
#from numpy.core.numeric import infty
!pip install numpy
!pip install pandas
!pip install plotly

!pip install scikit-learn # non-depreceated sklearn

ERROR: Invalid requirement: '#'


Task1

In [52]:
import pandas as pd
# read the csv and make it a pandas.DataFrame
frame = pd.DataFrame(pd.read_csv("./blood_transfusion.csv"))
frame.describe(include='all') # 18 columns, including 2 categorical: Month & VisitorType, and two boolean: Weekend & Revenue

,months_since_last_donation,total_number_of_donations,total_blood_donated,months_since_first_donation,class
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086,0.237968
std,8.095396,5.839307,1459.826781,24.376714,0.426124
min,0.000000,1.000000,250.000000,2.000000,0.000000
25%,2.750000,2.000000,500.000000,16.000000,0.000000
50%,7.000000,4.000000,1000.000000,28.000000,0.000000
75%,14.000000,7.000000,1750.000000,50.000000,0.000000
max,74.000000,50.000000,12500.000000,98.000000,1.000000


In [53]:
# visualise, understand, story

Task2

In [54]:
preprocessed = (frame.copy().dropna())
preprocessed = (preprocessed-preprocessed.min())/(preprocessed.max()-preprocessed.min()) # Normalization
print(preprocessed)

     months_since_last_donation  total_number_of_donations  \
0                      0.027027                   1.000000   
1                      0.000000                   0.244898   
2                      0.013514                   0.306122   
3                      0.027027                   0.387755   
4                      0.013514                   0.469388   
..                          ...                        ...   
743                    0.310811                   0.020408   
744                    0.283784                   0.020408   
745                    0.310811                   0.040816   
746                    0.527027                   0.000000   
747                    0.972973                   0.000000   

     total_blood_donated  months_since_first_donation  class  
0               1.000000                     1.000000    1.0  
1               0.244898                     0.270833    1.0  
2               0.306122                     0.343750    1.0  
3  

TASK 3

In [55]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train_s, X_test_s, Y_train_s, Y_test_s = train_test_split(
    preprocessed.drop('class', axis=1), preprocessed['class'], test_size=0.33)
X_train_l, X_test_l, Y_train_l, Y_test_l = train_test_split(
    preprocessed.drop('class', axis=1), preprocessed['class'], test_size=0.66)

Task 4

In [56]:
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score

#5.1 Confusion matrix
def confusionmatrix(predicted_y, real_y):
    tp, fp, tn, fn = 0, 0, 0, 0
    for index in range(len(predicted_y)):
        predicted = predicted_y.iloc[index]
        real = real_y.iloc[index]
        if predicted.item() == 1: 
            if predicted.item() == real.item(): 
                tp += 1
            else:
                fp += 1
        elif predicted.item() == 0:
            if predicted.item() == real.item(): 
                tn += 1
            else:
                fn += 1
    print(f"\t P \t\t N \n T \t {tp} \t {tn} \n F \t {fp} \t {fn}")

def accuraccy_checker(X_test, y_test, pred, mode="basic"):
    num, total = 0, 0
    predicted, true = [], []
    for index, row in X_test.iterrows():
        p = pred.predict(X_test.loc[[index]])
        q = list(y_test.loc[[index]])
        if p == q:
            num += 1
        total += 1
        predicted.append(p)
        true.append(q)

    if mode == "basic":
        print(f"Accuracy: {num/total*100:.2f}%")
    if mode == "classification_report":
        print(classification_report(true, predicted))
    if mode == "confusion_matrix":
        confusionmatrix(y_test, pred)
    if mode == "fbeta":
        print(fbeta_score(true, predicted, average='macro', beta=0.5))
        print(fbeta_score(true, predicted, average='weighted', beta=0.5))
        print(fbeta_score(true, predicted, average=None, beta=0.5))

# 4.1
#Task 4
#KNN
def knn_checker(predicted_y, real_y):
    num, total = 0, 0
    for index in range(len(predicted_y)):
        if list(predicted_y.iloc[index]) == (real_y.iloc[index]): 
            num += 1
        total += 1
    print(f"Accuracy: {num / total * 100:.2f}%")
    
def manhattan_distance(a, b):
    distance = 0
    for index, _ in enumerate(a):
        distance += ((a[index] - b[index]) ** 2) ** 0.5
    return distance      

def KNN_classifier(k, train_x, train_y, test_x):
    test_y = [0 for _ in range(test_x.shape[0])]
    test_y = pd.DataFrame(test_y)
    for index, (_,testrow) in enumerate(test_x.iterrows()):
        distances = []
        for _, trainrow in train_x.iterrows():
            testvalues = list(testrow)  
            trainvalues = list(trainrow)
            distance = manhattan_distance(testvalues, trainvalues)
            distances.append(distance)
        nearest_neighbours = sorted(range(len(distances)), key=lambda i: distances[i])[:k]
        neighbor_labels = train_y.iloc[nearest_neighbours]
        predicted_label = round(neighbor_labels.mean())
        test_y.iloc[index] = predicted_label
    return test_y

test_y_predicted = KNN_classifier(5, X_train_s, Y_train_s, X_test_s)        
knn_checker(test_y_predicted, Y_test_s)

# 4.2
from sklearn.naive_bayes import GaussianNB
nbc = GaussianNB()
nbc.fit(X_train_s, Y_train_s)
accuraccy_checker(X_test_s, Y_test_s, nbc)

# 4.3
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
svc = make_pipeline(StandardScaler(), SVC(gamma='auto'))
svc.fit(X_train_s, Y_train_s)
accuraccy_checker(X_test_s, Y_test_s, svc)

# 4.4
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=1, max_iter=250).fit(X_train_s, Y_train_s)
accuraccy_checker(X_test_s, Y_test_s, mlp)







Accuracy: 77.33%
Accuracy: 78.14%
Accuracy: 77.33%
Accuracy: 77.33%


In [57]:
accuraccy_checker(X_test_s, Y_test_s, nbc, mode = "confusion_matrix")

accuraccy_checker(X_test_s, Y_test_s, svc,  mode = "confusion_matrix")

accuraccy_checker(X_test_s, Y_test_s, mlp,  mode = "confusion_matrix")

accuraccy_checker(X_test_s, Y_test_s, nbc, mode = "classification_report")

accuraccy_checker(X_test_s, Y_test_s, svc,  mode = "classification_report")

accuraccy_checker(X_test_s, Y_test_s, mlp,  mode = "classification_report")

accuraccy_checker(X_test_s, Y_test_s, nbc, mode = "fbeta")

accuraccy_checker(X_test_s, Y_test_s, svc,  mode = "fbeta")

accuraccy_checker(X_test_s, Y_test_s, mlp,  mode = "fbeta")


AttributeError: 'GaussianNB' object has no attribute 'iloc'